Installed Spark

In [ ]:
!pip install pyspark

from pyspark import *
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
from pyspark.sql.functions import *
from pyspark.sql.types import *


sc = SparkContext.getOrCreate()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Define function to split each value

In [ ]:
def preprocess(line):
    id, movie_name, first_day_sell, first_week_sell, total_sell, avg_price, PV_watched, score, year = line.split(',')
    return id, movie_name, first_day_sell, first_week_sell, total_sell, avg_price, PV_watched, score, year

Import file

In [ ]:
from google.colab import files

In [ ]:
uploaded = files.upload() 
#If you haven't uploaded the data file, upload here. Remember to modify the path in next lie

Saving movie.csv to movie (1).csv


In [ ]:
rdd = sc.textFile('movie.csv')#read to rdd

#rdd.take(40)

['id,movie_name,first_day_sell,first_week_sell,total_sell,avg_price,PV_watched,score,year',
 '1,流浪地球2,47962.0,47962.0,402444.0,54.5,3126.8,9.3,2023',
 '2,满江红,40593.5,40539.5,454344.2,53.0,3367.0,9.4,2023',
 '3,无名,13744.4,54411.2,93057.0,44.0,1076.8,9.0,2023',
 '4,熊出没·伴我“熊芯”,13373.3,84653.3,148614.1,51.0,2405.4,9.2,2023',
 '5,交换人生,12727.0,30346.3,39345.6,53.0,1013.4,8.6,2023',
 '6,深海,7313.8,40748.4,91769.9,51.5,2413.6,9.0,2023',
 '7,冰雪与少年,195.0,3601.0,1.5,32.5,,,2023',
 '8,长津湖之水门桥,64111.9,253379.1,406458.4,57.4,1909.3,9.6,2022',
 '9,四海,22735.4,47425.1,54276.7,56.2,1008.5,8.6,2022',
 '10,这个杀手不太冷静,21428.8,139015.5,262795.5,56.0,2632.1,9.1,2022',
 '11,奇迹·笨小孩,18219.4,66873.7,137928.9,54.6,1511.9,9.5,2022',
 '12,熊出没·重返地球,10578.4,56267.6,97768.8,54.3,1660.4,9.4,2022',
 '13,狙击手,4826.6,26300.6,60205.9,53.5,1063.9,9.5,2022',
 '14,喜羊羊与灰太狼之筐出未来,2599.0,8701.7,16050.8,49.0,614.3,9.3,2022',
 '15,小虎墩大英雄,732.0,1743.0,2219.8,54.4,217.3,9.3,2022',
 '16,唐人街探案3,101123.2,257928.5,452341.1,49.5,2222.6,8.8,20

Delete the title of attributes

In [ ]:
header = rdd.first() #Titles
rdd = rdd.filter(lambda row:row != header) #delete titles

#rdd.take(1) #check operation result

['1,流浪地球2,47962.0,47962.0,402444.0,54.5,3126.8,9.3,2023']

In [ ]:
movie =rdd.map(preprocess)

#movie.take(1)

[('1',
  '流浪地球2',
  '47962.0',
  '47962.0',
  '402444.0',
  '54.5',
  '3126.8',
  '9.3',
  '2023')]

Trans movie to DataFrame, for further cleaning

In [ ]:
schema = StructType([
    StructField('id',StringType(),True),
    StructField('movie_name',StringType(),True),
    StructField('first_day_sell',StringType(),True),
    StructField('first_week_sell',StringType(),True),
    StructField('total_sell',StringType(),True),
    StructField('avg_price',StringType(),True),
    StructField('PV_watched',StringType(),True),
    StructField('score',StringType(),True),
    StructField('year',StringType(),True)
])

spark = SparkSession.builder.getOrCreate()
df = spark.createDataFrame(movie, schema)

Modify the data with "w" to pure numbers

In [ ]:
df = df.withColumn("first_day_sell", split(df["first_day_sell"], "w")[0])
df = df.withColumn("first_week_sell", split(df["first_week_sell"], "w")[0])
df = df.withColumn("total_sell", split(df["total_sell"], "w")[0])
df = df.withColumn("PV_watched", split(df["PV_watched"], "w")[0])

#df.toPandas()#function that show all results

id    movie_name first_day_sell first_week_sell total_sell avg_price  \
0    1         流浪地球2        47962.0         47962.0   402444.0      54.5   
1    2           满江红        40593.5         40539.5   454344.2      53.0   
2    3            无名        13744.4         54411.2    93057.0      44.0   
3    4    熊出没·伴我“熊芯”        13373.3         84653.3   148614.1      51.0   
4    5          交换人生        12727.0         30346.3    39345.6      53.0   
5    6            深海         7313.8         40748.4    91769.9      51.5   
6    7         冰雪与少年          195.0          3601.0        1.5      32.5   
7    8       长津湖之水门桥        64111.9        253379.1   406458.4      57.4   
8    9            四海        22735.4         47425.1    54276.7      56.2   
9   10      这个杀手不太冷静        21428.8        139015.5   262795.5      56.0   
10  11        奇迹·笨小孩        18219.4         66873.7   137928.9      54.6   
11  12      熊出没·重返地球        10578.4         56267.6    97768.8      54.3   
12  13           狙击手         4826.6         26300.6    60205.9      53.5   
13  14  喜羊羊与灰太狼之筐出未来         2599.0          8701.7    16050.8      49.0   
14  15        小虎墩大英雄          732.0          1743.0     2219.8      54.4   
15  16        唐人街探案3       101123.2        257928.5   452341.1      49.5   
16  17        你好，李焕英        29149.7        104970.4   541308.0      48.2   
17  18         刺杀小说家        13742.3         31411.0   103518.2      51.0   
18  19           侍神令         8112.7         16231.8    27397.6      49.0   
19  20      熊出没·狂野大陆         7268.7                    59529.2      46.2   
20  21      新神榜：哪吒重生         5132.7         13304.8    45634.2      49.1   
21  22          人潮汹涌         4321.5          7853.8    76234.5      46.2   
22  23         我为你喝彩           57.2           103.8     1575.0      36.7   
23  24         梦醒黄金城            2.2             3.3        5.9      29.3   
24  25     不要先生与好的女士         7801.0          7801.0       97.3      33.8   
25  26          流浪地球        19074.1        213168.0   468735.5      49.6   
26  27        疯狂的外星人        41018.7        146176.9   221321.2      45.3   
27  28          飞驰人生        32194.9        105386.7   172823.7      44.4   
28  29      熊出没·原始时代         7407.1         42495.1    71740.4      42.9   
29  30         新喜剧之王        27389.2         53614.3    62751.7      44.1   
30  31         神探蒲松龄         6647.5         12988.7    16099.3      43.7   
31  32       小猪佩奇过大年         5882.8         11306.6    12501.2      43.7   
32  33          廉政风云         5698.2          9778.8    12429.4      46.2   
33  34          红海行动        12951.1         46500.4   365020.6      42.2   
34  35        唐人街探案2        34076.3         99047.7   339776.6      39.3   
35  36          捉妖记2        54686.6        120447.0   223722.5      38.4   
36  37        西游记女儿国        16426.5         33149.5    72740.6      40.3   
37  38       熊出没·变形记         7612.8         20858.1    60551.7      37.0   
38  39         祖宗十九代         1742.2          4948.2    16985.2      37.8   
39  40       人怕出名猪怕壮           21.8            76.0      837.8      31.3   

   PV_watched score  year  
0      3126.8   9.3  2023  
1      3367.0   9.4  2023  
2      1076.8   9.0  2023  
3      2405.4   9.2  2023  
4      1013.4   8.6  2023  
5      2413.6   9.0  2023  
6                    2023  
7      1909.3   9.6  2022  
8      1008.5   8.6  2022  
9      2632.1   9.1  2022  
10     1511.9   9.5  2022  
11     1660.4   9.4  2022  
12     1063.9   9.5  2022  
13      614.3   9.3  2022  
14      217.3   9.3  2022  
15     2222.6   8.8  2021  
16     3434.0   9.5  2021  
17     1325.5   8.6  2021  
18     1006.7   8.5  2021  
19     1498.1   8.9  2021  
20     1491.2   8.8  2021  
21     1082.0   9.0  2021  
22                   2021  
23        9.4        2021  
24        5.5        2021  
25     1488.2   9.2  2019  
26     1332.2   8.5  2019  
27     1382.8   8.8  2019  
28      512.7   9.1  2019  
29      951.8   8.0  2019  
30      325.1   7.9

Modify the data type

In [ ]:
df = df.withColumn("id", df["id"].cast("integer"))
df = df.withColumn("first_day_sell", df["first_day_sell"].cast("decimal(10,1)"))
df = df.withColumn("first_week_sell", df["first_week_sell"].cast("decimal(10,1)"))
df = df.withColumn("total_sell", df["total_sell"].cast("decimal(10,1)"))
df = df.withColumn("avg_price", df["avg_price"].cast("decimal(10,1)"))
df = df.withColumn("PV_watched", df["PV_watched"].cast("decimal(10,1)"))
df = df.withColumn("score", df["score"].cast("decimal(10,1)"))
df = df.withColumn("year", df["year"].cast("integer"))

#df.toPandas() 

id    movie_name first_day_sell first_week_sell total_sell avg_price  \
0    1         流浪地球2        47962.0         47962.0   402444.0      54.5   
1    2           满江红        40593.5         40539.5   454344.2      53.0   
2    3            无名        13744.4         54411.2    93057.0      44.0   
3    4    熊出没·伴我“熊芯”        13373.3         84653.3   148614.1      51.0   
4    5          交换人生        12727.0         30346.3    39345.6      53.0   
5    6            深海         7313.8         40748.4    91769.9      51.5   
6    7         冰雪与少年          195.0          3601.0        1.5      32.5   
7    8       长津湖之水门桥        64111.9        253379.1   406458.4      57.4   
8    9            四海        22735.4         47425.1    54276.7      56.2   
9   10      这个杀手不太冷静        21428.8        139015.5   262795.5      56.0   
10  11        奇迹·笨小孩        18219.4         66873.7   137928.9      54.6   
11  12      熊出没·重返地球        10578.4         56267.6    97768.8      54.3   
12  13           狙击手         4826.6         26300.6    60205.9      53.5   
13  14  喜羊羊与灰太狼之筐出未来         2599.0          8701.7    16050.8      49.0   
14  15        小虎墩大英雄          732.0          1743.0     2219.8      54.4   
15  16        唐人街探案3       101123.2        257928.5   452341.1      49.5   
16  17        你好，李焕英        29149.7        104970.4   541308.0      48.2   
17  18         刺杀小说家        13742.3         31411.0   103518.2      51.0   
18  19           侍神令         8112.7         16231.8    27397.6      49.0   
19  20      熊出没·狂野大陆         7268.7            None    59529.2      46.2   
20  21      新神榜：哪吒重生         5132.7         13304.8    45634.2      49.1   
21  22          人潮汹涌         4321.5          7853.8    76234.5      46.2   
22  23         我为你喝彩           57.2           103.8     1575.0      36.7   
23  24         梦醒黄金城            2.2             3.3        5.9      29.3   
24  25     不要先生与好的女士         7801.0          7801.0       97.3      33.8   
25  26          流浪地球        19074.1        213168.0   468735.5      49.6   
26  27        疯狂的外星人        41018.7        146176.9   221321.2      45.3   
27  28          飞驰人生        32194.9        105386.7   172823.7      44.4   
28  29      熊出没·原始时代         7407.1         42495.1    71740.4      42.9   
29  30         新喜剧之王        27389.2         53614.3    62751.7      44.1   
30  31         神探蒲松龄         6647.5         12988.7    16099.3      43.7   
31  32       小猪佩奇过大年         5882.8         11306.6    12501.2      43.7   
32  33          廉政风云         5698.2          9778.8    12429.4      46.2   
33  34          红海行动        12951.1         46500.4   365020.6      42.2   
34  35        唐人街探案2        34076.3         99047.7   339776.6      39.3   
35  36          捉妖记2        54686.6        120447.0   223722.5      38.4   
36  37        西游记女儿国        16426.5         33149.5    72740.6      40.3   
37  38       熊出没·变形记         7612.8         20858.1    60551.7      37.0   
38  39         祖宗十九代         1742.2          4948.2    16985.2      37.8   
39  40       人怕出名猪怕壮           21.8            76.0      837.8      31.3   

   PV_watched score  year  
0      3126.8   9.3  2023  
1      3367.0   9.4  2023  
2      1076.8   9.0  2023  
3      2405.4   9.2  2023  
4      1013.4   8.6  2023  
5      2413.6   9.0  2023  
6        None  None  2023  
7      1909.3   9.6  2022  
8      1008.5   8.6  2022  
9      2632.1   9.1  2022  
10     1511.9   9.5  2022  
11     1660.4   9.4  2022  
12     1063.9   9.5  2022  
13      614.3   9.3  2022  
14      217.3   9.3  2022  
15     2222.6   8.8  2021  
16     3434.0   9.5  2021  
17     1325.5   8.6  2021  
18     1006.7   8.5  2021  
19     1498.1   8.9  2021  
20     1491.2   8.8  2021  
21     1082.0   9.0  2021  
22       None  None  2021  
23        9.4  None  2021  
24        5.5  None  2021  
25     1488.2   9.2  2019  
26     1332.2   8.5  2019  
27     1382.8   8.8  2019  
28      512.7   9.1  2019  
29      951.8   8.0  2019  
30      325.1   7.9

Replace null data with 0

In [ ]:
df = df.na.fill(value=0.0)
#df.toPandas()

id    movie_name first_day_sell first_week_sell total_sell avg_price  \
0    1         流浪地球2        47962.0         47962.0   402444.0      54.5   
1    2           满江红        40593.5         40539.5   454344.2      53.0   
2    3            无名        13744.4         54411.2    93057.0      44.0   
3    4    熊出没·伴我“熊芯”        13373.3         84653.3   148614.1      51.0   
4    5          交换人生        12727.0         30346.3    39345.6      53.0   
5    6            深海         7313.8         40748.4    91769.9      51.5   
6    7         冰雪与少年          195.0          3601.0        1.5      32.5   
7    8       长津湖之水门桥        64111.9        253379.1   406458.4      57.4   
8    9            四海        22735.4         47425.1    54276.7      56.2   
9   10      这个杀手不太冷静        21428.8        139015.5   262795.5      56.0   
10  11        奇迹·笨小孩        18219.4         66873.7   137928.9      54.6   
11  12      熊出没·重返地球        10578.4         56267.6    97768.8      54.3   
12  13           狙击手         4826.6         26300.6    60205.9      53.5   
13  14  喜羊羊与灰太狼之筐出未来         2599.0          8701.7    16050.8      49.0   
14  15        小虎墩大英雄          732.0          1743.0     2219.8      54.4   
15  16        唐人街探案3       101123.2        257928.5   452341.1      49.5   
16  17        你好，李焕英        29149.7        104970.4   541308.0      48.2   
17  18         刺杀小说家        13742.3         31411.0   103518.2      51.0   
18  19           侍神令         8112.7         16231.8    27397.6      49.0   
19  20      熊出没·狂野大陆         7268.7             0.0    59529.2      46.2   
20  21      新神榜：哪吒重生         5132.7         13304.8    45634.2      49.1   
21  22          人潮汹涌         4321.5          7853.8    76234.5      46.2   
22  23         我为你喝彩           57.2           103.8     1575.0      36.7   
23  24         梦醒黄金城            2.2             3.3        5.9      29.3   
24  25     不要先生与好的女士         7801.0          7801.0       97.3      33.8   
25  26          流浪地球        19074.1        213168.0   468735.5      49.6   
26  27        疯狂的外星人        41018.7        146176.9   221321.2      45.3   
27  28          飞驰人生        32194.9        105386.7   172823.7      44.4   
28  29      熊出没·原始时代         7407.1         42495.1    71740.4      42.9   
29  30         新喜剧之王        27389.2         53614.3    62751.7      44.1   
30  31         神探蒲松龄         6647.5         12988.7    16099.3      43.7   
31  32       小猪佩奇过大年         5882.8         11306.6    12501.2      43.7   
32  33          廉政风云         5698.2          9778.8    12429.4      46.2   
33  34          红海行动        12951.1         46500.4   365020.6      42.2   
34  35        唐人街探案2        34076.3         99047.7   339776.6      39.3   
35  36          捉妖记2        54686.6        120447.0   223722.5      38.4   
36  37        西游记女儿国        16426.5         33149.5    72740.6      40.3   
37  38       熊出没·变形记         7612.8         20858.1    60551.7      37.0   
38  39         祖宗十九代         1742.2          4948.2    16985.2      37.8   
39  40       人怕出名猪怕壮           21.8            76.0      837.8      31.3   

   PV_watched score  year  
0      3126.8   9.3  2023  
1      3367.0   9.4  2023  
2      1076.8   9.0  2023  
3      2405.4   9.2  2023  
4      1013.4   8.6  2023  
5      2413.6   9.0  2023  
6         0.0   0.0  2023  
7      1909.3   9.6  2022  
8      1008.5   8.6  2022  
9      2632.1   9.1  2022  
10     1511.9   9.5  2022  
11     1660.4   9.4  2022  
12     1063.9   9.5  2022  
13      614.3   9.3  2022  
14      217.3   9.3  2022  
15     2222.6   8.8  2021  
16     3434.0   9.5  2021  
17     1325.5   8.6  2021  
18     1006.7   8.5  2021  
19     1498.1   8.9  2021  
20     1491.2   8.8  2021  
21     1082.0   9.0  2021  
22        0.0   0.0  2021  
23        9.4   0.0  2021  
24        5.5   0.0  2021  
25     1488.2   9.2  2019  
26     1332.2   8.5  2019  
27     1382.8   8.8  2019  
28      512.7   9.1  2019  
29      951.8   8.0  2019  
30      325.1   7.9

Filter movies with total_sell>100

In [ ]:
df = df.filter(df["total_sell"]>100.0)
#df.toPandas()

id    movie_name first_day_sell first_week_sell total_sell avg_price  \
0    1         流浪地球2        47962.0         47962.0   402444.0      54.5   
1    2           满江红        40593.5         40539.5   454344.2      53.0   
2    3            无名        13744.4         54411.2    93057.0      44.0   
3    4    熊出没·伴我“熊芯”        13373.3         84653.3   148614.1      51.0   
4    5          交换人生        12727.0         30346.3    39345.6      53.0   
5    6            深海         7313.8         40748.4    91769.9      51.5   
6    8       长津湖之水门桥        64111.9        253379.1   406458.4      57.4   
7    9            四海        22735.4         47425.1    54276.7      56.2   
8   10      这个杀手不太冷静        21428.8        139015.5   262795.5      56.0   
9   11        奇迹·笨小孩        18219.4         66873.7   137928.9      54.6   
10  12      熊出没·重返地球        10578.4         56267.6    97768.8      54.3   
11  13           狙击手         4826.6         26300.6    60205.9      53.5   
12  14  喜羊羊与灰太狼之筐出未来         2599.0          8701.7    16050.8      49.0   
13  15        小虎墩大英雄          732.0          1743.0     2219.8      54.4   
14  16        唐人街探案3       101123.2        257928.5   452341.1      49.5   
15  17        你好，李焕英        29149.7        104970.4   541308.0      48.2   
16  18         刺杀小说家        13742.3         31411.0   103518.2      51.0   
17  19           侍神令         8112.7         16231.8    27397.6      49.0   
18  20      熊出没·狂野大陆         7268.7             0.0    59529.2      46.2   
19  21      新神榜：哪吒重生         5132.7         13304.8    45634.2      49.1   
20  22          人潮汹涌         4321.5          7853.8    76234.5      46.2   
21  23         我为你喝彩           57.2           103.8     1575.0      36.7   
22  26          流浪地球        19074.1        213168.0   468735.5      49.6   
23  27        疯狂的外星人        41018.7        146176.9   221321.2      45.3   
24  28          飞驰人生        32194.9        105386.7   172823.7      44.4   
25  29      熊出没·原始时代         7407.1         42495.1    71740.4      42.9   
26  30         新喜剧之王        27389.2         53614.3    62751.7      44.1   
27  31         神探蒲松龄         6647.5         12988.7    16099.3      43.7   
28  32       小猪佩奇过大年         5882.8         11306.6    12501.2      43.7   
29  33          廉政风云         5698.2          9778.8    12429.4      46.2   
30  34          红海行动        12951.1         46500.4   365020.6      42.2   
31  35        唐人街探案2        34076.3         99047.7   339776.6      39.3   
32  36          捉妖记2        54686.6        120447.0   223722.5      38.4   
33  37        西游记女儿国        16426.5         33149.5    72740.6      40.3   
34  38       熊出没·变形记         7612.8         20858.1    60551.7      37.0   
35  39         祖宗十九代         1742.2          4948.2    16985.2      37.8   
36  40       人怕出名猪怕壮           21.8            76.0      837.8      31.3   

   PV_watched score  year  
0      3126.8   9.3  2023  
1      3367.0   9.4  2023  
2      1076.8   9.0  2023  
3      2405.4   9.2  2023  
4      1013.4   8.6  2023  
5      2413.6   9.0  2023  
6      1909.3   9.6  2022  
7      1008.5   8.6  2022  
8      2632.1   9.1  2022  
9      1511.9   9.5  2022  
10     1660.4   9.4  2022  
11     1063.9   9.5  2022  
12      614.3   9.3  2022  
13      217.3   9.3  2022  
14     2222.6   8.8  2021  
15     3434.0   9.5  2021  
16     1325.5   8.6  2021  
17     1006.7   8.5  2021  
18     1498.1   8.9  2021  
19     1491.2   8.8  2021  
20     1082.0   9.0  2021  
21        0.0   0.0  2021  
22     1488.2   9.2  2019  
23     1332.2   8.5  2019  
24     1382.8   8.8  2019  
25      512.7   9.1  2019  
26      951.8   8.0  2019  
27      325.1   7.9  2019  
28      463.2   6.0  2019  
29      369.4   7.2  2019  
30     2498.6   9.4  2018  
31     2118.4   9.0  2018  
32     2996.1   8.2  2018  
33     1701.6   7.9  2018  
34     1462.1   9.1  2018  
35      798.4   7.7  2018  
36       12.7   8.0  2018

Preprocessing is finished, save the result to a csv file

In [ ]:
file=r"content\movie"
df.repartition(1).write.csv(path=file, header=True, sep="," , encoding="utf-8", mode = "overwrite")